In [1]:
import synapseclient
import pandas as pd
import json
import csv

In [2]:
def TapPerformance(TapPD1):
    TapPD1.loc[TapPD1.TappedButtonId == 'TappedButtonLeft', 'TappedButtonId'] = 0
    TapPD1.loc[TapPD1.TappedButtonId == 'TappedButtonRight', 'TappedButtonId'] = 1
    TapPD1.loc[TapPD1.TappedButtonId == 'TappedButtonNone', 'TappedButtonId'] = 2
    #TapPD1.TappedButtonId[TapPD1.TappedButtonId == 'TappedButtonLeft'] = 0
    #TapPD1.TappedButtonId[TapPD1.TappedButtonId == 'TappedButtonRight'] = 1
    #TapPD1.TappedButtonId[TapPD1.TappedButtonId == 'TappedButtonNone'] = 2
    TapPD1 = Artifactfixed(TapPD1)
    TapPerf= TapPD1.shape[0]
    return (TapPerf)

In [3]:
def ConvertTypePatient(MedTime):
    if (MedTime == "I don't take Parkinson medications"):
        ClassPD = 0
    elif (MedTime == "Immediately before Parkinson medication"):
        ClassPD = 1
    elif (MedTime == "Just after Parkinson medication (at your best)"):
        ClassPD =2
    else (MedTime == "Another time"):
        ClassPD =3

SyntaxError: invalid syntax (<ipython-input-3-03a0ddc291f5>, line 8)

In [4]:
#Index=list()
def Artifactfixed(TapCont):
    Ind =  [True] * TapCont.shape[0]
    for item in range(0,TapCont.shape[0]-1):
        if TapCont.TapCoordinate[item]==TapCont.TapCoordinate[item+1]:
            if TapCont.TappedButtonId[item]==2:
                Ind[item]=False
            elif TapCont.TappedButtonId[item+1]==2:
                Ind[item]=False
    TapContArtFixed=TapCont[Ind]
    return TapContArtFixed
          

In [5]:

syn = synapseclient.Synapse()
syn.login('catarivera8', 'Crm1061709701')


## QUERY THE mPower PROJECT (syn4993293) FOR ALL OF THE TABLES
# See examples: https://python-docs.synapse.org/build/html/index.html 
# and https://www.synapse.org/#!Synapse:syn4993293/discussion/threadId=6785
with open('TappingTable.csv') as csvfile:
    TapTable=pd.read_csv(csvfile, delimiter=',')
with open('DemoTable.csv') as csvfile:
    DemoTable=pd.read_csv(csvfile, delimiter=',') 
#Using Patient column (healthCode) as index for the Data frames
#TapTableB = TapTable.set_index("healthCode", drop = False) 
#DemoTableB = DemoTable.set_index("healthCode", drop = False)    

      
#TapPD = list()
PathList = list()
LabelList = list()
#PatientType = list()
#PatientList = list()
#AgeList = list()
Perform =list()
for offset in range(0,800,100):
    results = syn.tableQuery('SELECT * FROM syn5511439 LIMIT 100 OFFSET '+str(offset))
    file_map=syn.downloadTableColumns(results,['tapping_results.json.TappingSamples'])
    for file_handle_id, path in file_map.items():
        #TapPD.append(pd.read_json(path))
        PathList.append(path)
        LabelList.append(int(file_handle_id))
        #PatientType.append(TapTableB['medTimepoint'][TapTable['tapping_results.json.TappingSamples'] == int(file_handle_id)])
        Perform.append(TapPerformance(pd.read_json(path))) #Tap performance
        #PatientList.append(TapTableB['healthCode'][TapTableB['tapping_results.json.TappingSamples'] == int(file_handle_id)])
        #DemoTableB.loc[(TapTableB['healthCode'][TapTableB['tapping_results.json.TappingSamples'] == int(file_handle_id)]),"age"]
        

Welcome, Catalina Rivera!



In [8]:
#Creating directory
d = {'tapping_results.json.TappingSamples':LabelList,'TapPerform':Perform}
PerfInfo = pd.DataFrame(d)
#Merges TapTables with the PerfInfo and drops thos values in TapTable that are not defined in PerInfo
TapTable=TapTable.merge(PerfInfo,left_on='tapping_results.json.TappingSamples', right_on='tapping_results.json.TappingSamples')
#Sub-DataFrame of Demo Table with only features of interest: Age, diagnostic year, gender.
DemoTable.head()
DemoTableN=DemoTable[["age","healthCode","diagnosis-year","gender"]]
DemoTableN.shape
#Merges TapTables with the PerfInfo and drops thos values in TapTable that are not defined in PerInfo
TapTableN=TapTable.merge(DemoTableN)
print(TapTableN['TapPerform'])

0      294
1      289
2      164
3      229
4      176
      ... 
650    194
651    265
652    264
653    241
654    581
Name: TapPerform, Length: 655, dtype: int64


In [9]:
#df2.loc["California", : ]
ListTemp=list()
ListTemp.append(DemoTableB.loc[(TapTableB['healthCode'][TapTableB['tapping_results.json.TappingSamples'] == int(file_handle_id)]),"age"])
print(ListTemp)
#print(DemoTableB.loc(str([TapTableB.loc['healthCode'][TapTableB['tapping_results.json.TappingSamples'] == int(file_handle_id)]])),"age")
#DemoTableB.loc([TapTableB[healthCode'][TapTableB['tapping_results.json.TappingSamples'] == int(file_handle_id)]][0],'age')

[healthCode
7a837f11-98c5-422f-8b27-0653a6b6a47e    32.0
Name: age, dtype: float64]


In [71]:
with open('DemoTable.csv') as csvfile:
    DemoTable=pd.read_csv(csvfile, delimiter=',')  
    

DemoTableB = DemoTable.set_index("healthCode", drop = False)
print(DemoTable.head())
print(DemoTableB.head())
#print((DemoTable['healthCode'])==(PatientList[0]))
#print((PatientList[0]))
#print((DemoTable['healthCode']))
#print( (PatientList[0]) == "639e8a78-3631-4231-bda1-c911c1b169e5")

   ROW_ID  ROW_VERSION                              recordId  \
0   20580           21  bbb003a9-5c7b-4d66-a1d3-bcd1430edbb7   
1   20581           21  889f09c7-e5fa-41d3-8beb-e6d0e6c85100   
2   20582           21  05258304-77f7-4761-bc58-52dc743e44cd   
3   20583           21  b331ff56-c6d9-430a-bc6c-b6f52d2d8e9f   
4   20584           21  6926995b-1d3e-45c9-858b-bd5e0ceb4e68   

                             healthCode     createdOn            appVersion  \
0  639e8a78-3631-4231-bda1-c911c1b169e5  1.425900e+12  version 1.0, build 7   
1  9295f618-177c-4676-b6aa-dc8419fd37ec  1.425920e+12  version 1.0, build 7   
2  52fe366a-2a9f-4260-9fb1-0fbc637a6cf4  1.425930e+12  version 1.0, build 7   
3  67bdd316-26fc-4fc7-8431-bf9f41a649dd  1.425930e+12  version 1.0, build 7   
4  45b4e2ca-8d15-4736-828c-829e3d4177f4  1.425930e+12  version 1.0, build 7   

         phoneInfo   age are-caretaker deep-brain-stimulation  ...  \
0         iPhone 6  24.0         False                  False  ...   


In [34]:
with open('TappingTable.csv') as csvfile:
    TapTable=pd.read_csv(csvfile, delimiter=',')
with open('TappingTable.csv') as csvfile:
    TapTable=pd.read_csv(csvfile, delimiter=',')


7800


,ROW_ID,ROW_VERSION,recordId,healthCode,createdOn,appVersion,phoneInfo,accel_tapping.json.items,tapping_results.json.ButtonRectLeft,tapping_results.json.ButtonRectRight,tapping_results.json.endDate,tapping_results.json.startDate,tapping_results.json.TappingSamples,tapping_results.json.TappingViewSize,medTimepoint
0,82315,18,caf7a74b-0d2e-4c43-9233-ed5dc6e7b176,639e8a78-3631-4231-bda1-c911c1b169e5,1.425900e+12,"version 1.0, build 7",iPhone 6,5395782.0,"{{0, 191.5}, {104, 104}}","{{128, 191.5}, {104, 104}}",1.425900e+12,1.425900e+12,5395798.0,"{375, 603}",I don't take Parkinson medications
1,82316,18,94c95572-b405-49ca-9eb0-7426254990d5,52fe366a-2a9f-4260-9fb1-0fbc637a6cf4,1.425930e+12,"version 1.0, build 7",iPhone 5s (GSM),5403976.0,"{{0, 156.5}, {104, 104}}","{{128, 156.5}, {104, 104}}",1.425930e+12,1.425930e+12,5403991.0,"{320, 504}",I don't take Parkinson medications
2,82317,18,ebd99144-5b84-486e-818c-5f8daf8beebf,67bdd316-26fc-4fc7-8431-bf9f41a649dd,1.425930e+12,"version 1.0, build 7",iPhone 6 Plus,5397281.0,"{{0, 191.33333333333331}, {104, 104}}","{{128, 191.33333333333331}, {104, 104}}",1.425930e+12,1.425930e+12,5397298.0,"{414, 672}",I don't take Parkinson medications
3,82318,18,8e32cff2-c8f9-4047-af57-411a98fc9116,67bdd316-26fc-4fc7-8431-bf9f41a649dd,1.425930e+12,"version 1.0, build 7",iPhone 6 Plus,5395191.0,"{{0, 191.33333333333331}, {104, 104}}","{{128, 191.33333333333331}, {104, 104}}",1.425930e+12,1.425930e+12,5395211.0,"{414, 672}",I don't take Parkinson medications
4,82319,18,69851454-05b2-44f1-9248-299ca684b801,be43841c-d034-4705-88da-87e3536163b2,1.425930e+12,"version 1.0, build 7",iPhone 6,5390401.0,"{{0, 189}, {104, 104}}","{{128, 189}, {104, 104}}",1.425930e+12,1.425930e+12,5390413.0,"{375, 603}",I don't take Parkinson medications


In [20]:
df = pd.DataFrame({'A' : [5,6,3,4], 'B' : [1,2,3, 5]})
df

,A,B
0,5,1
1,6,2
2,3,3
3,4,5


In [22]:
df[df['A'] == 3]

,A,B
2,3,3


In [16]:
2+2

4